# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import yaml

import feisty

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [14]:
def _comparison(matlab_vals, py_vals, row_name, full_table, seps, thres):
    nstep = np.min([len(matlab_vals["time"].data), len(py_vals["time"].data)])
    nx = np.min([len(matlab_vals["X"].data), len(py_vals["X"].data)])
    matlab_vals = matlab_vals.isel(time=slice(0, nstep), X=slice(0, nx)).data
    py_vals = py_vals.isel(time=slice(0, nstep), X=slice(0, nx)).data
    if thres:
        thres_mask = np.logical_and(np.abs(py_vals) > thres, np.abs(matlab_vals) > thres)
        py_vals = np.where(thres_mask, py_vals, 0)
        matlab_vals = np.where(thres_mask, matlab_vals, 0)
    rel_err_denom = np.where(
        matlab_vals != 0, matlab_vals, 1
    )  # avoid dividing by 0 in next np.where() statement
    if np.any(np.isnan(py_vals)):
        print(f"python has nans for {rowname}!")
        return
    rel_errs = np.where(matlab_vals != 0, np.abs((py_vals - matlab_vals) / rel_err_denom), np.nan)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals == 0), 0, rel_errs)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals > 0), np.inf, rel_errs)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals < 0), -np.inf, rel_errs)

    if full_table or rel_err > 1e-12:
        max_inds = np.nonzero(np.abs(rel_errs) == np.max(np.abs(rel_errs)))
        t_ind = max_inds[0][0]
        x_ind = max_inds[1][0]
        matlab_val = matlab_vals[t_ind][x_ind]
        py_val = py_vals[t_ind][x_ind]
        rel_err = rel_errs[t_ind][x_ind]
        print(
            f"{seps[0]}{row_name} (t={t_ind}, X={x_ind}){seps[1]}{matlab_val:10.4e}{seps[2]}"
            + f"{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}"
        )


def compare_nc(baseline_ds, test_da_or_ds, full_table=True, markdown_formatting="true", thres=None):
    if type(test_da_or_ds) == xr.Dataset:
        ds = test_da_or_ds
        da = None
        baseline_da = None
        table_header = ''
    else:
        da = test_da_or_ds
        ds = None
        da_dims = ["group", "fish"]
        da_dim = ''
        for dad in da_dims:
            if dad in da.dims:
                da_dim = dad
                break
        baseline_da = baseline_ds[da.name]
        table_header = da_dim
    if markdown_formatting:
        print(f"| {table_header} | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ["| ", " | ", " | ", " |"]
    else:
        seps = ["", ": ", ", ", ""]
    if da is not None:
        for n, dimname in enumerate(da[da_dim]):
            _comparison(
                baseline_da.isel({da_dim: n}),
                da.isel({da_dim: n}),
                dimname.data,
                full_table,
                seps,
                thres,
            )
    else:
        for varname in ds:
            _comparison(
                baseline_ds.isel(zooplankton=0)[varname],
                ds.isel(zooplankton=0)[varname],
                varname,
                full_table,
                seps,
                thres,
            )

In [4]:
# matlab_script = "test_case" ; force_nonnegative=True
# matlab_script, force_nonnegative = "test_locs3", True
matlab_script, force_nonnegative = "FOSI_cesm", False

baselines_from_nc = xr.open_dataset(f'../matlab_baselines/{matlab_script}.nc')

nyears = 1
settings_in = {}
if matlab_script == "test_case":
    testcase = feisty.config_testcase("tanh_shelf", "cyclic")
elif matlab_script in ["test_locs3", "FOSI_cesm_daily", "FOSI_cesm"]:
    is_locs3 = matlab_script == "test_locs3"
    if is_locs3:
        start_date = '0001-01-01'
        nyears = 2
    else:
        # only changing benthic_prey.defaults.carrying_capacity, food_web[0]['preference'], food_web[1]['preference'], and food_web[2]['preference']
        # To say this UI needs work is an understatement
        start_date = '0249-01-01'
        settings_in['benthic_prey'] = {
            'defaults': {'benthic_efficiency': 0.075, 'carrying_capacity': 80},
            'members': [{'name': 'benthic_prey'}],
        }
        settings_in['food_web'] = [
            {'predator': 'Sf', 'prey': 'Zoo', 'preference': 0.9},
            {'predator': 'Sp', 'prey': 'Zoo', 'preference': 0.9},
            {'predator': 'Sd', 'prey': 'Zoo', 'preference': 0.9},
            {'predator': 'Mf', 'prey': 'Zoo', 'preference': 0.45},
            {'predator': 'Mf', 'prey': 'Sf', 'preference': 1.0},
            {'predator': 'Mf', 'prey': 'Sp', 'preference': 1.0},
            {'predator': 'Mf', 'prey': 'Sd', 'preference': 1.0},
            {'predator': 'Mp', 'prey': 'Zoo', 'preference': 0.45},
            {'predator': 'Mp', 'prey': 'Sf', 'preference': 1.0},
            {'predator': 'Mp', 'prey': 'Sp', 'preference': 1.0},
            {'predator': 'Mp', 'prey': 'Sd', 'preference': 1.0},
            {'predator': 'Md', 'prey': 'benthic_prey', 'preference': 1.0},
            {'predator': 'Lp', 'prey': 'Mf', 'preference': 0.5},
            {'predator': 'Lp', 'prey': 'Mp', 'preference': 1.0},
            {'predator': 'Lp', 'prey': 'Md', 'preference': 1.0},
            {'predator': 'Ld', 'prey': 'Mf', 'preference': 0.375},
            {'predator': 'Ld', 'prey': 'Mp', 'preference': 0.75},
            {'predator': 'Ld', 'prey': 'Md', 'preference': 1.0},
            {'predator': 'Ld', 'prey': 'benthic_prey', 'preference': 1.0},
        ]
    testcase = feisty.config_from_netcdf(
        './forcing.yaml',
        matlab_script,
        start_date=start_date,
        ignore_year_in_forcing=is_locs3,  # True,
        settings_in=settings_in,
    )
else:
    raise ValueError(f"unknown matlab_script '{matlab_script}'")

# Set negative forcing values to zero
if force_nonnegative:
    for var in ['poc_flux_bottom', 'zooC', 'zoo_mort']:
        baselines_from_nc[var].data = np.where(
            baselines_from_nc[var].data > 0, baselines_from_nc[var].data, 0
        )

In [5]:
if testcase.ignore_year:
    new_forcing = testcase.forcing.isel(time=slice(1, -1)).transpose('zooplankton', 'time', 'X')
else:
    new_forcing = testcase.forcing.transpose('zooplankton', 'time', 'X')
compare_nc(
    baselines_from_nc,
    new_forcing,
)

print(
    f"\nMax diff in depth: {np.max(np.abs(baselines_from_nc['dep'].data - testcase.obj.domain_dict['bathymetry'].data))}"
)

|  | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| T_pelagic (t=5, X=5317) | 6.8918e-07 | -1.2154e+00 | 1.7635e+06 |
| T_bottom (t=297, X=6919) | -2.7372e-07 | 2.0280e-01 | 7.4092e+05 |
| poc_flux_bottom (t=0, X=7569) | 0.0000e+00 | 8.5357e-02 |        inf |
| zooC (t=0, X=650) | 0.0000e+00 | 2.7150e+00 |        inf |
| zoo_mort (t=0, X=11246) | 0.0000e+00 | 1.2717e-01 |        inf |

Max diff in depth: 85.20895599999972


## Run the model


In [6]:
nsteps = nyears * 365
make_plot = True
%time testcase.run(nsteps)

CPU times: user 4min 54s, sys: 20min 35s, total: 25min 29s
Wall time: 1h 37min 26s


In [15]:
compare_nc(baselines_from_nc, testcase.ds["biomass"], nsteps, thres=1e-15)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf (t=11, X=15633) | 1.1178e-03 | 2.1996e-02 | 1.8678e+01 |
| Sp (t=10, X=15633) | 8.6116e-05 | 1.1066e-03 | 1.1850e+01 |
| Sd (t=10, X=15633) | 2.9038e-05 | 3.6319e-04 | 1.1508e+01 |
| Mf (t=229, X=11677) | 2.2930e-05 | 7.8569e-06 | 6.5735e-01 |
| Mp (t=301, X=10407) | 1.0511e-09 | 4.8833e-10 | 5.3543e-01 |
| Md (t=217, X=11677) | 1.5760e-04 | 4.2251e-04 | 1.6809e+00 |
| Lp (t=346, X=45523) | 8.1850e-12 | 3.6667e-13 | 9.5520e-01 |
| Ld (t=364, X=45524) | 5.0344e-01 | 4.8942e-01 | 2.7858e-02 |
| benthic_prey (t=364, X=76989) | 6.2653e-02 | 1.5234e-01 | 1.4314e+00 |


In [ ]:
if nsteps > 1 and make_plot:
    isel_dict = {"time": range(len(testcase.ds.time))}
    baselines_tmp = baselines_from_nc.isel(isel_dict).assign_coords(
        {'group': testcase.ds.group.data, 'time': testcase.ds.time.data}
    )
    fig, axs = plt.subplots(1, 2, figsize=(15, 4))
    if matlab_script == "test_case":
        X = 0
        ylim = [1e-6, 1]
    if matlab_script == "test_locs3":
        X = 0
        ylim = [1e-6, 100]
    if matlab_script == "FOSI_cesm":
        X = 55000  # looks good
        # X = 15633  # large error in small classes
        # X = 11677  # large error in medium classes
        # X = 76989  # large error in benthic
        ylim = [5e-7, 50]
    for ngroup, group in enumerate(testcase.ds.group.data):
        testcase.ds.biomass.sel(group=group).isel(X=X).plot(ax=axs[0])
        baselines_tmp.biomass.sel(group=group).isel(X=X).plot(ax=axs[1])
        # try:
        #     baselines_from_nc.biomass.sel(group=group).isel(X=X).plot(ax=axs[1])
        # except:
        #     try:
        #         baselines_from_nc.biomass.sel(group=group.encode('UTF-8')).isel(X=X).plot(ax=axs[1])
        #     except:
        #         baselines_from_nc.biomass.isel(group=ngroup).isel(X=X).plot(ax=axs[1])
    for ax in axs:
        ax.set_ylim(ylim)
        ax.set_yscale("log")
    axs[0].set_title("python")
    axs[1].set_title("matlab")
    axs[1].set_ylabel("")
    axs[1].set_yticklabels("")
    plt.legend(testcase.ds.group.data, bbox_to_anchor=(1.025, 0.5), loc=6)
    fig.suptitle(f"Comparison at X={X}")

In [ ]:
if nsteps > 1 and make_plot:
    fig, axs = plt.subplots(1, 2, figsize=(15, 4))
    for ngroup, group in enumerate(testcase.ds.group.data):
        da1 = testcase.ds.biomass.sel(group=group).isel(X=X)
        da2 = baselines_tmp.biomass.sel(group=group).isel(X=X)
        (da1 - da2).plot(ax=axs[0])
        (np.abs(da1 - da2) / da2).plot(ax=axs[1])
    axs[0].set_title("absolute error")
    axs[0].set_ylabel("")
    axs[1].set_title("relative error")
    axs[1].set_ylabel("")
    axs[1].set_yscale("log")
    plt.legend(testcase.ds.group.data, bbox_to_anchor=(1.025, 0.5), loc=6)
    plt.title("Relative Error")
    fig.suptitle(f"Comparison at X={X}")

In [10]:
compare_nc(baselines_from_nc.isel(X=[X]), testcase.ds["biomass"].isel(X=[X]), nsteps, thres=1e-15)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf (t=364, X=0) | 6.7385e-03 | 6.4911e-03 | 3.6704e-02 |
| Sp (t=364, X=0) | 2.4204e-06 | 2.3345e-06 | 3.5493e-02 |
| Sd (t=364, X=0) | 3.3327e-05 | 3.2329e-05 | 2.9936e-02 |
| Mf (t=298, X=0) | 2.1324e+00 | 2.1326e+00 | 9.9782e-05 |
| Mp (t=314, X=0) | 9.8662e-04 | 9.8668e-04 | 6.8555e-05 |
| Md (t=297, X=0) | 1.2150e-02 | 1.2151e-02 | 7.5675e-05 |
| Lp (t=314, X=0) | 2.2667e-02 | 2.2673e-02 | 2.5694e-04 |
| Ld (t=364, X=0) | 2.1164e-01 | 2.1165e-01 | 6.8773e-06 |
| benthic_prey (t=364, X=0) | 2.0469e+01 | 2.0469e+01 | 4.3050e-05 |
